# Projet maison n° 3

In [6]:
# imports
import pandas as pd
import re
import glob

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [7]:
# names us


def df_names_us():
    FileName=[]
    FileName=['names/yob'+str(l)+'.txt' for l in range(1880,2022)]
    SD=re.compile('[^a-zA-Z./]+')
    d=pd.DataFrame([])
        
    for Name in FileName:
             l=SD.search(Name).group()
             dtmp=pd.read_csv(Name,names=['name','gender','births'],keep_default_na=False)   
             dtmp=pd.concat([pd.DataFrame([l for i in range(len(dtmp))],columns=['year']),dtmp],axis=1)
             d=pd.concat([d,dtmp],ignore_index=True)
             d.reset_index(drop=True)
                
    d=d.sort_values(by=['year','gender','births'],ascending=[False,True,False],ignore_index=True) 
   
    return d

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [14]:
# names fr
def df_names_fr():
        File='nat2021.csv'
        df=pd.read_csv(File,delimiter=';',names=['gender','name','year','births'],header=0,keep_default_na=False)
        col=df.columns.tolist()
        df=df[[col[2],col[1],col[0],col[3]]]
        Map={'1':'H','2':'F'}
        df['gender']=df['gender'].map({1:'M',2:'F'})
        df.drop(df[df['name'].astype(str).apply(len)<2].index,inplace=True)
        df.drop(df[df['name']=='_PRENOMS_RARES'].index,inplace=True)
        df.drop(df[df['year']=='XXXX'].index,inplace=True)
        df.reset_index(inplace=True, drop=True)
        df['name']=df['name'].str.title()
        df=df.sort_values(by=['year','gender','births'],ascending=[False,True,False],ignore_index=True) 
        return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [15]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    return df

### Tests

In [16]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [17]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [18]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... FAIL

FAIL: test_df_taux_change (__main__.Session3Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-16-349d947a4711>", line 36, in test_df_taux_change
    self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
AssertionError: False is not true

----------------------------------------------------------------------
Ran 3 tests in 7.107s

FAILED (failures=1)
